## Meta matching v1.0
This jupyter notebook demonstrates you how to load and use meta-matching algorthm. In this demonstration, we performed Meta-matching (DNN) stacking with 100 example subjects.

Package needed (and version this jupyter notebook tested):
* Numpy (1.16.4)
* Scipy (1.0.0)
* PyTorch (1.3.1)
* Scikit-learn (0.21.3)

### Step 0. Setup
Please modify the `path_repo` below to your repo position:

In [1]:
path_repo = '../'#'/home/the/deepGround/code/2002/Meta_matching_models/'

In [3]:
# initialization and random seed set

import os
import scipy
import torch
import random
import sklearn
import numpy as np
import sys

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

import warnings
warnings.filterwarnings("ignore")

Please modify the gpu number here if you want to use different gpu. If the gpu you assigned not availiable, it will assign to cpu

In [4]:
gpu = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Step 1. load data
Load the example data that we provided, it contains 
* Example input functional connectivity (FC) `x` with size of (100, 87571)
    * 100 is number of subjects
    * 87571 is flatten vector of 419 by 419 FC (419*418/2=87571)
    * We perform subject-wise z-normalization, using mean and std of 87571 elements of each subject
* Example output phenotypes `y` with size of (100, 3)
    * 3 is number of phenotypes.

In [5]:
path_v11 = os.path.join(path_repo, 'v1.1')
path_v10 = os.path.join(path_repo, 'v1.0')
sys.path.append(path_v10)
from CBIG_model_pytorch import demean_norm

npz = os.path.join(path_v11, 'meta_matching_v1.1_data.npz')
npz = np.load(npz)
print(npz.files)
x_input = npz['x']
y_input = npz['y']
x_input = demean_norm(x_input)
print(x_input.shape, y_input.shape)

['x', 'y']
(100, 87571) (100, 3)


### Step 2. Split data
Here, we also split 100 subjects to 80/20, where 80 for training, and 20 for test.

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_input, y_input, test_size=0.2, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(80, 87571) (20, 87571) (80, 3) (20, 3)


### Step 3. Prepare data for PyTorch model
Then we prepare data for DNN model, we will input both the `x_train` and `x_test` into the model to get the predicted phenotypes. 

For meta-matching (DNN) stacking, we do not need real phenotype for the DNN model, I created all zeros `y_dummy` just for function requirement. In some other cases, like meta-matching (DNN) finetuning, you need to use real phenotype data here.

In [7]:
from torch.utils.data import DataLoader
from CBIG_model_pytorch import multi_task_dataset

batch_size = 16
y_dummy = np.zeros(y_train.shape)
dset_train = multi_task_dataset(x_train, y_dummy, True)
trainLoader = DataLoader(dset_train,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=1)

y_dummy = np.zeros(y_test.shape)
dset_test = multi_task_dataset(x_test, y_dummy, True)
testLoader = DataLoader(dset_test,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=1)

### Step 4. load model
Here we load the meta-matching model saved, it is a DNN that takes FC as input and output 67 phenotypes prediction trained on 67 UK Biobank phenotypes

In [8]:
path_model_weight = os.path.join(path_v11, 'meta_matching_v1.1_model.pkl_torch') 
net = torch.load(path_model_weight, map_location=device)
net.to(device)
net.train(False)
print(net)

dnn(
  (layers): ModuleList(
    (0): Sequential(
      (0): Dropout(p=0.3, inplace=False)
      (1): Linear(in_features=87571, out_features=256, bias=True)
      (2): ReLU()
      (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Dropout(p=0.3, inplace=False)
      (1): Linear(in_features=256, out_features=256, bias=True)
      (2): ReLU()
      (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Dropout(p=0.3, inplace=False)
      (1): Linear(in_features=256, out_features=256, bias=True)
      (2): ReLU()
      (3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): Sequential(
      (0): Dropout(p=0.3, inplace=False)
      (1): Linear(in_features=256, out_features=67, bias=True)
    )
  )
)


### Step 5. DNN model predict
Here we apply the DNN trained on 67 UK Biobank phenotypes to predict the 67 phenotypes on `x_train` and `x_test`. We will get the predicted 67 phenotypes on both 80 training subjects and 20 test subjects.

In [9]:
y_train_pred = np.zeros((0, 67))
for (x, _) in trainLoader:
    x= x.to(device)
    outputs = net(x)
    y_train_pred = np.concatenate((y_train_pred, outputs.data.cpu().numpy()), axis=0)
print(y_train_pred.shape, '\n', y_train_pred)

y_test_pred = np.zeros((0, 67))
for (x, _) in testLoader:
    x= x.to(device)
    outputs = net(x)
    y_test_pred = np.concatenate((y_test_pred, outputs.data.cpu().numpy()), axis=0)
print(y_test_pred.shape, '\n', y_test_pred)

(80, 67) 
 [[-0.15115939 -0.0669269  -0.54655945 ... -0.11759652 -0.50293493
  -0.37641898]
 [ 0.29037341 -0.20854312  0.23705281 ...  0.15359323 -0.07533579
  -0.01452626]
 [ 0.10086952  0.20367797 -0.53936934 ... -0.04685985 -0.14448746
  -0.0352214 ]
 ...
 [ 0.05406483 -0.03227098  0.01010649 ...  0.10382769  0.06453447
   0.05130197]
 [-0.02054616 -0.17987496  0.16705912 ...  0.1099993  -0.22622883
  -0.34170136]
 [-0.10911851  0.1235078  -0.43411684 ... -0.07488919 -0.53739721
  -0.25947312]]
(20, 67) 
 [[-0.05762977 -0.17934641  0.16057274 ...  0.16511196 -0.55663419
  -0.25849211]
 [ 0.01560813 -0.1447289   0.19979201 ...  0.13954791  0.01221326
  -0.41223368]
 [ 0.05661141 -0.21707436  0.19259959 ...  0.1661057  -0.24444117
  -0.16643339]
 ...
 [-0.01382111 -0.127198    0.20016842 ...  0.12121768 -0.32253277
   0.0576634 ]
 [-0.15039177 -0.21929857  0.01748148 ...  0.12292583 -0.37921733
  -0.48630238]
 [ 0.15801965  0.05928475 -0.45958063 ... -0.11260507  0.15486403
  -0.21450

### Step 6. Stacking
Perform stacking with `y_train_pred`, `y_test_pred`, `y_train`, where we use the prediction of 80 subjects `y_train_pred` (input) and real data `y_train` (output) to train the stacking model (you can either use all 67 source phenotypes for stacking, or select top K source phenotypes relevant to the target phenotype, like we mentioned in our paper; it turns out that these 2 ways achieves similar performances), then we applied the model to `y_test_pred` to get final prediction of 3 phenotypes on 20 subjects.

#### Hyperparameter Tuning 
In `stacking()` function, we set the range of `alpha` as `[0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 10, 15, 20]`. You are weclomed to modify the range of `alpha` to get better performance on your own data.

In [10]:
from CBIG_model_pytorch import stacking
y_test_final=np.zeros((y_test_pred.shape[0], y_train.shape[1]))
for i in range(y_train.shape[1]):
    # For each test phenotype, perform stacking by developing a KRR model
    y_test_temp, _ = stacking(y_train_pred, y_test_pred, y_train[:,i].view(), [0.00001, 0.0001, 0.001, 0.004, 0.007, 0.01, 0.04, 0.07, 0.1, 0.4, 0.7, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5, 10, 15, 20])
    y_test_final[:,i] = y_test_temp.flatten()
print(y_test_final.shape, '\n', y_test_final)

(20, 3) 
 [[59.91704316 25.72586686 26.07246047]
 [66.14376526 23.98923972 24.73108215]
 [61.25782014 24.89340214 22.11663259]
 [21.40825319 13.06584165  9.37246313]
 [74.82713639 27.95233725 28.73392918]
 [49.82987774 18.36064402 19.47435966]
 [36.49521893 21.86627855 29.7535562 ]
 [34.89049406  8.73729406 13.03381664]
 [49.96686959 16.23559303 17.04143582]
 [60.55911895 23.84655006 25.25979719]
 [26.22655036 13.15274099 15.6317502 ]
 [90.25412737 39.1989077  41.48038924]
 [44.38907628 18.84119275 18.80375602]
 [69.74869482 31.5875928  34.68293162]
 [52.18927948 22.5219697  29.531368  ]
 [61.18475632 17.90580022 27.18276641]
 [39.63361043 19.58916493 20.62720972]
 [66.41021447 20.73182542 18.82197354]
 [46.6794769  20.7430434  24.27900141]
 [52.1961148  13.92751791 15.36534066]]


### Step 7. Evaluation
Evaluate the prediction performance.

In [11]:
from scipy.stats.stats import pearsonr
corr = np.zeros((y_train.shape[1]))
for i in range(y_train.shape[1]):
    corr[i] = pearsonr(y_test_final[:, i], y_test[:, i])[0]
print(corr)

[0.6493213  0.53661207 0.25953581]


### Step 8. Haufe transform predictive network features (PNFs) computation
Here we compute the PNF for stacking we just performed. It computes the covariance between 3 phenotype prediciton and each element of FC on the 80 training subjects. The final PNF is in shape of (87571, 3), where 87571 is number of 419 by 419 FC elements, and 3 is number of phenotypes.

In [12]:
from CBIG_model_pytorch import covariance_rowwise

y_train_haufe, _ = stacking(y_train_pred, y_train_pred, y_train)
print(y_train_haufe.shape)
cov = covariance_rowwise(x_train, y_train_haufe)
print(cov, '\n', cov.shape)

(80, 3)
[[-9.51272534e-04 -3.33093629e-04  9.99491542e-04]
 [-4.95032426e-04 -1.34704160e-05 -2.96746334e-04]
 [-3.60040114e-03 -1.20511813e-03 -1.83170320e-03]
 ...
 [ 9.54923023e-04  1.62136461e-04  8.06833900e-04]
 [ 1.61852164e-03  7.27061452e-04  1.63064681e-03]
 [ 1.80749311e-03  8.30753940e-04  4.38313184e-04]] 
 (87571, 3)


### Step 9. Haufe transform predictive network features (PNFs) computation for training phenotypes
Here we compute the PNF for stacking we just performed. It computes the covariance between 3 phenotype prediciton and each training phenotypes on the 80 training subjects. The final PNF is in shape of (67, 3), where 67 is number of training phenotypes, and 3 is number of phenotypes.

In [13]:
from CBIG_model_pytorch import covariance_rowwise

cov = covariance_rowwise(y_train_pred, y_train_haufe)
print(cov, '\n', cov.shape)

[[ 1.14476059e-01  4.24958531e-02 -2.56837861e-02]
 [-2.35807282e-01 -1.43320430e-01 -4.63684846e-03]
 [ 8.79415505e-01  4.66306778e-01  1.75841287e-01]
 [ 1.05367594e-01  6.90872062e-02  2.35385686e-02]
 [-1.46605681e-02 -3.73209161e-02 -1.80059741e-01]
 [-1.21257278e-01 -1.50802049e-01 -3.29285698e-01]
 [ 1.24786394e-01  8.16866955e-02  1.78748205e-01]
 [ 9.03748222e-02  5.98972858e-02  2.07077366e-01]
 [ 5.15225829e-01  2.60515380e-01  1.05632279e-01]
 [-9.27633893e-02 -9.72508105e-02 -1.83468128e-01]
 [ 3.83940713e-01  2.00519881e-01  1.50639053e-01]
 [-4.30634539e-01 -2.88612709e-01 -1.17827291e-01]
 [-1.37145572e-01 -8.56516379e-02 -1.07273467e-01]
 [ 7.31631508e-01  4.57041168e-01  7.08423710e-01]
 [ 3.56652530e-01  1.78622558e-01  2.17740663e-01]
 [ 2.22915358e+00  1.29261675e+00  7.02672001e-01]
 [-5.11638474e-01 -2.58350469e-01 -2.42740146e-01]
 [-9.79363826e-02  4.49926335e-02 -1.86941683e-01]
 [-7.45365024e-01 -4.00650668e-01 -4.72234137e-01]
 [-7.88092020e-02 -4.22677762e-